In [1]:
import numpy as np
from MFEA_lib.tasks.function import Sphere, Ackley, Rosenbrock
from MFEA_lib.model.mfea import MFEA1
from MFEA_lib.operators.CrossOver import SBX_CrossOver
from MFEA_lib.operators.Selection import ElitismSelection
from MFEA_lib.operators.Mutation import Polynomial_Mutation


In [2]:
tasks = [Sphere(30, bound= [-100, 100]), 
        Ackley(50, bound= [-100, 100]), 
        Rosenbrock(20, bound= [-100, 100]), 
        Rosenbrock(30, bound= [-100, 100])
]


In [3]:
model = MFEA1()
model.compile(
    cross_over= SBX_CrossOver(nc = 15),
    mutation= Polynomial_Mutation(nm = 15),
    selection= ElitismSelection()
)
sol, his = model.fit(tasks= tasks, num_generations= 2000, num_inds_each_task= 100, rmp =0.3)

Epoch [100/2000], [=>                  ]   5% ,func_val: [5071.423921782715, 20.294823540074095, 70795840.03730945, 156737765.15446123]

Epoch [200/2000], [==>                 ]  10% ,func_val: [91.68085603067578, 20.109877703854412, 25587.114298712237, 45708.74296777359]

Epoch [300/2000], [===>                ]  15% ,func_val: [11.053539188409916, 20.04223196176817, 417.92831141582906, 1406.3810082325613]

Epoch [400/2000], [====>               ]  20% ,func_val: [1.8870444690640233, 20.01983095379461, 92.85360947697622, 206.40967004017472]

Epoch [500/2000], [=====>              ]  25% ,func_val: [0.7240821318738768, 20.009494511510276, 49.76180526094903, 86.80814821006466]

Epoch [600/2000], [======>             ]  30% ,func_val: [0.2760595531879007, 20.004542179245, 31.20670879118325, 52.94426450343554]

Epoch [700/2000], [=======>            ]  35% ,func_val: [0.05989694350105013, 20.003100198439206, 16.79266632958505, 34.65423029891557]

Epoch [800/2000], [========>           ]  